In [22]:
import pandas as pd
melb = pd.read_csv('data/melbfor3.csv')

→ Сводная таблица принимает на вход данные из отдельных столбцов и группирует их. В результате получается новая таблица, которая позволяет увидеть многомерное обобщение данных. Таким образом, благодаря сводным таблицам мы можем оценить зависимость между двумя и более признаками данных.

МЕТОД GROUPBY КАК СПОСОБ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ

На самом деле мы с вами уже строили простейшие одномерные сводные таблицы с помощью метода groupby — мы рассматривали сводную таблицу в контексте группировки по одному признаку. 

Например, мы уже умеем строить таблицу, которая показывает зависимость медианной цены и площади здания от числа комнат:

In [23]:
melb.groupby('Rooms')[['Price', 'BuildingArea']].median()

,Price,BuildingArea
Rooms,,
1,385000.0,107.0
2,690000.0,126.0
3,950000.0,126.0
4,1285000.0,142.0
5,1660000.0,176.0
6,1800000.0,126.0
7,1496000.0,216.5
8,1515000.0,126.0
10,900000.0,126.0


Также можно построить таблицу, в которой мы будем учитывать не только число комнат, но и тип здания (Type). Для этого в параметрах метода groupby() укажем список из нескольких интересующих нас столбцов.

In [24]:
melb.groupby(['Rooms', 'Type'])['Price'].mean()

Rooms  Type     
1      house        8.668655e+05
       townhouse    5.927045e+05
       unit         3.899289e+05
2      house        1.017238e+06
       townhouse    7.101585e+05
       unit         6.104905e+05
3      house        1.109233e+06
       townhouse    9.847087e+05
       unit         8.505963e+05
4      house        1.462283e+06
       townhouse    1.217092e+06
       unit         1.037476e+06
5      house        1.877327e+06
       townhouse    1.035000e+06
6      house        1.869508e+06
       unit         5.200000e+05
7      house        1.920700e+06
8      house        1.510286e+06
       unit         2.250000e+06
10     house        9.000000e+05
Name: Price, dtype: float64

В результате выполнения такого кода мы получаем Series, которая обладает несколькими уровнями индексов: первый уровень — число комнат, второй уровень — тип здания. Такая организация индексов называется иерархической. Вычисление параметра (средней цены) происходит во всех возможных комбинациях признаков.

Для того, чтобы финальный результат был представлен в виде сводной таблицы (первый группировочный признак по строкам, а второй — по столбцам), а не в виде Series с иерархическими индексами, к результату чаще всего применяют метод unstack(), который позволяет переопределить вложенный индекс в виде столбцов таблицы:

In [25]:
melb.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

Type,house,townhouse,unit
Rooms,,,
1,8.668655e+05,5.927045e+05,3.899289e+05
2,1.017238e+06,7.101585e+05,6.104905e+05
3,1.109233e+06,9.847087e+05,8.505963e+05
4,1.462283e+06,1.217092e+06,1.037476e+06
5,1.877327e+06,1.035000e+06,NaN
6,1.869508e+06,NaN,5.200000e+05
7,1.920700e+06,NaN,NaN
8,1.510286e+06,NaN,2.250000e+06
10,9.000000e+05,NaN,NaN


МЕТОД PIVOT_TABLE ДЛЯ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ

На самом деле метод groupby редко используется при двух параметрах, так как для построения сводных таблиц существует специальный и более простой метод — pivot_table().

values — имя столбца, по которому необходимо получить сводные данные, применяя агрегирующую функцию;
index — имя столбца, значения которого станут строками сводной таблицы;
columns — имя столбца, значения которого станут столбцами сводной таблицы;
aggfunc — имя или список имён агрегирующих функций (по умолчанию — подсчёт среднего, 'mean');
fill_value — значение, которым необходимо заполнить пропуски (по умолчанию пропуски не заполняются).

In [26]:
melb.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

Type,house,townhouse,unit
Rooms,,,
1,866866.0,592705.0,389929.0
2,1017238.0,710158.0,610491.0
3,1109233.0,984709.0,850596.0
4,1462283.0,1217092.0,1037476.0
5,1877327.0,1035000.0,0.0
6,1869508.0,0.0,520000.0
7,1920700.0,0.0,0.0
8,1510286.0,0.0,2250000.0
10,900000.0,0.0,0.0


Несложно понять, что метод pivot_table() имеет преимущество перед группировкой по нескольким критериям. Оно заключается в наличии специальных аргументов для строк и столбцов сводной таблицы, благодаря чему уменьшается вероятность запутаться при построении более сложных (многомерных) сводных таблиц, о которых мы поговорим далее.

А теперь давайте проанализируем продажи в каждом из регионов в зависимости от того, будний был день или выходной. Для этого построим сводную таблицу, в которой строками будут являться названия регионов (Regionname), а в столбцах будет располагаться наш «признак-мигалка» выходного дня (Weekend)

In [27]:
melb.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)

Weekend,0,1
Regionname,,
Eastern Metropolitan,447,1024
Eastern Victoria,13,40
Northern Metropolitan,1258,2632
Northern Victoria,11,30
South-Eastern Metropolitan,123,327
Southern Metropolitan,1534,3161
Western Metropolitan,960,1988
Western Victoria,8,24


Разберём ещё один пример: найдём, как зависит средняя и медианная площадь участка (Landsize) от типа объекта (Type) и его региона (Regionname). Чтобы посмотреть несколько статистических параметров, нужно передать в аргумент aggfunc список из агрегирующих функций.

In [28]:
melb.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

median                          mean              \
Type                        house townhouse   unit        house   townhouse   
Regionname                                                                    
Eastern Metropolitan        674.0     233.5  203.0   717.422847  269.440678   
Eastern Victoria            843.0       0.0  230.0  3108.960000    0.000000   
Northern Metropolitan       459.5     134.0    0.0   619.249092  317.325733   
Northern Victoria           724.0       0.0    0.0  3355.463415    0.000000   
South-Eastern Metropolitan  630.5     240.0  199.0   664.306701  212.160000   
Southern Metropolitan       586.0     246.0    0.0   569.643881  278.858824   
Western Metropolitan        531.0     198.0   62.0   507.883406  244.560669   
Western Victoria            599.5       0.0    0.0   655.500000    0.000000   

                                        
Type                              unit  
Regionname                              
Eastern Metropolitan        330.444444  
Eastern Victoria            295.333333  
Northern Metropolitan       495.026538  
Northern Victoria             0.000000  
South-Eastern Metropolitan  357.864865  
Southern Metropolitan       466.380245  
Western Metropolitan        557.637232  
Western Victoria              0.000000

МНОГОМЕРНЫЕ СВОДНЫЕ ТАБЛИЦЫ

Для того чтобы исследовать зависимость от большего числа признаков, можно передать список признаков в параметр index или параметр columns.

Давайте построим таблицу, в которой по индексам будут располагаться признаки метода продажи (Method) и типа объекта (Type), по столбцам — наименование региона (Regionname), а на пересечении строк и столбцов будет стоять медианная цена объекта (Price):

In [29]:
melb.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

Regionname        Eastern Metropolitan  Eastern Victoria  \
Method Type                                                
PI     house                 1244000.0          780000.0   
       townhouse              760000.0               0.0   
       unit                   650000.0               0.0   
S      house                 1127000.0          675000.0   
       townhouse              828000.0               0.0   
       unit                   645750.0          492000.0   
SA     house                  932500.0          950000.0   
       townhouse              807500.0               0.0   
       unit                        0.0               0.0   
SP     house                 1050000.0          672500.0   
       townhouse              910000.0               0.0   
       unit                   515000.0          400000.0   
VB     house                 1100000.0          712500.0   
       townhouse              892500.0               0.0   
       unit                   500000.0               0.0   

Regionname        Northern Metropolitan  Northern Victoria  \
Method Type                                                  
PI     house                   900000.0           500000.0   
       townhouse               632500.0                0.0   
       unit                    410000.0                0.0   
S      house                   920000.0           555000.0   
       townhouse               750000.0                0.0   
       unit                    525500.0                0.0   
SA     house                   817500.0           540000.0   
       townhouse               425000.0                0.0   
       unit                    616000.0                0.0   
SP     house                   900000.0           521000.0   
       townhouse               690000.0                0.0   
       unit                    470000.0                0.0   
VB     house                  1050000.0           690000.0   
       townhouse               640000.0                0.0   
       unit                    450000.0                0.0   

Regionname        South-Eastern Metropolitan  Southern Metropolitan  \
Method Type                                                           
PI     house                        865000.0              1725000.0   
       townhouse                   1190000.0              1055000.0   
       unit                         525000.0               571250.0   
S      house                        883300.0              1611000.0   
       townhouse                    875000.0              1135000.0   
       unit                         606000.0               655000.0   
SA     house                        880000.0              1390000.0   
       townhouse                         0.0              1141000.0   
       unit                              0.0               580000.0   
SP     house                        770000.0              1521750.0   
       townhouse                    800000.0              1162500.0   
       unit                         601000.0               550000.0   
VB     house                        850000.0              1800000.0   
       townhouse                         0.0              1250000.0   
       unit                         700000.0               500000.0   

Regionname        Western Metropolitan  Western Victoria  
Method Type                                               
PI     house                  870000.0          630000.0  
       townhouse              670000.0               0.0  
       unit                   360000.0               0.0  
S      house                  870000.0          397500.0  
       townhouse              729000.0               0.0  
       unit                   489000.0               0.0  
SA     house                  772500.0               0.0  
       townhouse              467500.0               0.0  
       unit                   571000.0               0.0  
SP     house                  865000.0          360000.0  
       t

ДОСТУП К ДАННЫМ В СВОДНОЙ ТАБЛИЦЕ

?
Как получить доступ к данным или произвести фильтрацию в сложной сводной таблице, где есть дополнительные индексы?

Давайте рассмотрим, что собой представляют столбцы сложной сводной таблицы.

Запишем сводную таблицу, которую мы создавали ранее в переменную pivot:

In [30]:
pivot = melb.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)
pivot.columns

MultiIndex([('median',     'house'),
            ('median', 'townhouse'),
            ('median',      'unit'),
            (  'mean',     'house'),
            (  'mean', 'townhouse'),
            (  'mean',      'unit')],
           names=[None, 'Type'])

В результате мы получаем объект MultiIndex. Этот объект хранит в себе шесть комбинаций пар столбцов (два статистических параметра и три типа здания), то есть есть шесть возможных вариантов обращения к столбцам таблицы.

Мультииндексы раскрываются подобно вложенным словарям — по очереди, как матрёшка. Чтобы получить доступ к определённому столбцу, вы должны сначала обратиться к столбцу, который находится уровнем выше.

In [31]:
pivot['mean']['unit']

Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

Аналогично производится и фильтрация данных. Например, если нам нужны регионы, в которых средняя площадь здания для домов типа house меньше их медианной площади, то мы можем найти их следующим образом:

In [32]:
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

median                        mean              \
Type                   house townhouse  unit       house   townhouse   
Regionname                                                             
Southern Metropolitan  586.0     246.0   0.0  569.643881  278.858824   
Western Metropolitan   531.0     198.0  62.0  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

Чтобы получить индексы отфильтрованной таблицы, можно воспользоваться атрибутом index и обернуть результат в список:

In [33]:
list(filtered_pivot.index)

['Southern Metropolitan', 'Western Metropolitan']

Примечание. На самом деле мультииндексные таблицы можно создавать и вручную. Давайте посмотрим на синтаксис данной конструкции:

In [34]:
import numpy as np
mser = pd.Series(
    np.random.rand(8),
	index=[['white','white','white','blue','blue','red','red','red'], 
           ['up','down','right','up','down','up','down','left']])
display(mser)

white  up       0.772026
       down     0.500554
       right    0.011538
blue   up       0.807367
       down     0.875648
red    up       0.796831
       down     0.132314
       left     0.339921
dtype: float64

В данном примере мы создаём объект Series со вложенными индексами. Мы передаём в качестве индексов Series вложенный список, где первый список задаёт внешний уровень вложенности, а второй список — внутренний уровень вложенности. Значения Series — случайные числа от 0 до 1, сгенерированные функцией np.random.rand() (ваши значения могут отличаться).

Аналогично создаются DataFrame со вложенными признаками (вложенными столбцами) — для этого вложенный список передаётся в параметр columns при инициализации таблицы:

In [35]:
mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
display(mframe)

pen               paper          
                   1         2         1         2
white up    0.156497 -0.540546  0.508115 -0.856156
      down  0.116601  0.018773 -0.620161  0.582375
red   up    0.517105 -0.982079 -0.920215  0.151727
      down -0.218956 -1.907053 -1.059410  0.492554

In [ ]:
x = melb.pivot_table(
    values='BuildingArea',
    index='Rooms',
    columns='Type',
    aggfunc='median',
    fill_value=0
)
x